In [1]:
 

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'



In [3]:
!ls

clean_for_subb.ipynb  offenseval-training-v1.tsv


In [51]:
df = pd.read_csv("offenseval-training-v1.tsv", sep ='\t')

In [52]:
df.head()


,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [53]:
df.rename({ 'subtask_a':'target', 'subtask_b':'subb', 'subtask_c':'subc'}, axis=1,inplace=True)


In [54]:
z = {'NOT' : 0 ,'OFF' : 1}
df['target'] = df.target.map(z)
z = {'UNT' : 0 ,'TIN' : 1}
df['subb'] = df.subb.map(z)

In [55]:
'''not-0 off-1'''

'not-0 off-1'

In [56]:
'''unt-0 tin-1'''

'unt-0 tin-1'

In [57]:
df.head()

,id,tweet,target,subb,subc
0,86426,@USER She should ask a few native Americans wh...,1,0.0,NaN
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1,1.0,IND
2,16820,Amazon is investigating Chinese employees who ...,0,NaN,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",1,0.0,NaN
4,43605,@USER @USER Obama wanted liberals &amp; illega...,0,NaN,NaN


In [58]:
df.drop(['subc'],axis=1,inplace=True)

In [59]:
df.head()

,id,tweet,target,subb
0,86426,@USER She should ask a few native Americans wh...,1,0.0
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1,1.0
2,16820,Amazon is investigating Chinese employees who ...,0,NaN
3,62688,"@USER Someone should'veTaken"" this piece of sh...",1,0.0
4,43605,@USER @USER Obama wanted liberals &amp; illega...,0,NaN


In [60]:
df2=pd.DataFrame(df[df.target == 1])

In [61]:
df2.head()

,id,tweet,target,subb
0,86426,@USER She should ask a few native Americans wh...,1,0.0
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1,1.0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",1,0.0
5,97670,@USER Liberals are all Kookoo !!!,1,1.0
6,77444,@USER @USER Oh noes! Tough shit.,1,0.0


In [62]:
df2.drop(['target'],axis=1,inplace=True)

In [63]:
df2.head()

,id,tweet,subb
0,86426,@USER She should ask a few native Americans wh...,0.0
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1.0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",0.0
5,97670,@USER Liberals are all Kookoo !!!,1.0
6,77444,@USER @USER Oh noes! Tough shit.,0.0


In [26]:
df2.subb.value_counts()

1.0    3876
0.0     524
Name: subb, dtype: int64

In [27]:
'''with @user'''

'with @user'

In [28]:


import re

from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
part3 = r'#[A-Za-z0-9_]+'
combined_pat = r'|'.join((pat1, pat2,part3))
www_pat = r'www.[^ ]+'
url = r'URL'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner(text):
    
    try:
        bom_removed = text.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = text
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(url, '', stripped)
    
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()



In [29]:
df2.head()

,id,tweet,subb
0,86426,@USER She should ask a few native Americans wh...,0.0
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1.0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",0.0
5,97670,@USER Liberals are all Kookoo !!!,1.0
6,77444,@USER @USER Oh noes! Tough shit.,0.0


In [30]:
testing = df2.tweet

In [31]:


test_result = []
for t in testing:
    test_result.append(tweet_cleaner(str(t)))



In [32]:
len(test_result)

4400

In [33]:



clean_df = pd.DataFrame(df2.id,columns=['id'])
clean_df['text'] = test_result
clean_df['target'] = df2.subb


clean_df.head()

,id,text,target
0,86426,she should ask few native americans what their...,0.0
1,90194,go home you re drunk,1.0
3,62688,someone should vetaken this piece of shit to v...,0.0
5,97670,liberals are all kookoo,1.0
6,77444,oh noes tough shit,0.0


In [34]:



clean_df.to_csv('train_clean_subb_withoutuser.csv',index=False,encoding='utf-8')




In [35]:
'''with @user'''

'with @user'

In [79]:


import re

from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()

#pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
part3 = r'#[A-Za-z0-9_]+'
combined_pat = r'|'.join((pat2,part3))
www_pat = r'www.[^ ]+'
url = r'URL'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}
neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def tweet_cleaner(text):
    
    try:
        bom_removed = text.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = text
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    stripped = re.sub(url, '', stripped)
    
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()



In [80]:
df2.head()

,id,tweet,subb
0,86426,@USER She should ask a few native Americans wh...,0.0
1,90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,1.0
3,62688,"@USER Someone should'veTaken"" this piece of sh...",0.0
5,97670,@USER Liberals are all Kookoo !!!,1.0
6,77444,@USER @USER Oh noes! Tough shit.,0.0


In [81]:
testing = df2.tweet

In [82]:


test_result = []
for t in testing:
    test_result.append(tweet_cleaner(str(t)))



In [83]:
len(test_result)

4400

In [85]:



clean_df1 = pd.DataFrame(df2.id,columns=['id'])
clean_df1['text'] = test_result
clean_df1['target'] = df2.subb


clean_df1.head()

,id,text,target
0,86426,user she should ask few native americans what ...,0.0
1,90194,user user go home you re drunk user,1.0
3,62688,user someone should vetaken this piece of shit...,0.0
5,97670,user liberals are all kookoo,1.0
6,77444,user user oh noes tough shit,0.0


In [86]:



clean_df1.to_csv('train_clean_subb_withuser.csv',index=False,encoding='utf-8')


